In [33]:
# coding=utf-8
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import json

dir_root = "D:\\WorkSpace\\Python\\Taobao-live-product-identification"
dir_name = "D:\\WorkSpace\\Python\\Taobao-live-product-identification\\Live_data"
dir_dataset = os.listdir(dir_name)
for f in dir_dataset:
    print(dir_name + os.sep + f)

# 这里是因为我只下载了part1,为了后面可以加上另外五个part,所以加上一个for
# for i,f in enumerate(dir_dataset):
f = dir_dataset[0]
dir_six_parts = os.listdir(dir_name + os.sep + f)
dir_six_parts.sort(reverse=False)

dir_image = dir_name + os.sep + f + os.sep + dir_six_parts[0]
dir_image_annotation = dir_name + os.sep + f + os.sep + dir_six_parts[1]
dir_image_text = dir_name + os.sep + f + os.sep + dir_six_parts[2]
dir_video = dir_name + os.sep + f + os.sep + dir_six_parts[3]
dir_video_annotation = dir_name + os.sep + f + os.sep + dir_six_parts[4]
dir_video_frame = dir_name + os.sep + f + os.sep + dir_six_parts[5]
dir_video_text = dir_name + os.sep + f + os.sep + dir_six_parts[6]

D:\WorkSpace\Python\Taobao-live-product-identification\Live_data\Live_demo_20200117


In [34]:
# 首先把json都弄出来处理成一个df 每一个json就是一条信息
image_json_dic_list = []
for img_id in os.listdir(dir_image_annotation):
    dir_fold = dir_image_annotation + os.sep + img_id
    jsons = os.listdir(dir_fold)
    for i,img_json in enumerate(jsons):
        f = open(dir_fold + os.sep + img_json,'r')
        json_file = json.load(f)
        json_file['img_path'] = dir_image + os.sep + img_id + os.sep + str(i) + ".jpg"
        json_file['img_text_path'] = dir_image_text + os.sep + img_id + '.txt'
        image_json_dic_list.append(json_file)
# 开始将字典list转化为纯list
image_json_list = []
for i in image_json_dic_list:
    l = []
    if len(i['annotations']) != 0:
        for j in range(0,len(i['annotations'])):
            if len(i['annotations'][j]['box']) == 4:
                l = [i['item_id'],i['img_name'],
                 i['annotations'][j]['label'],
                 i['annotations'][j]['viewpoint'],
                 i['annotations'][j]['display'],
                 i['annotations'][j]['instance_id'],
                 i['annotations'][j]['box'][0],
                 i['annotations'][j]['box'][1],
                 i['annotations'][j]['box'][2],
                 i['annotations'][j]['box'][3],
                 i['img_path'],i['img_text_path']]
            elif len(i['annotations'][j]['box']) == 0:
                l = [i['item_id'],i['img_name'],
                 i['annotations'][j]['label'],
                 i['annotations'][j]['viewpoint'],
                 i['annotations'][j]['display'],
                 i['annotations'][j]['instance_id'],
                 None,None,None,None,
                 i['img_path'],i['img_text_path']]
    elif len(i['annotations']) == 0:
        l = [i['item_id'],i['img_name'],
             None,None,None,None,
             None,None,None,None,
             i['img_path'],i['img_text_path']]
    image_json_list.append(l)
# 将list转化为df 存入csv中
df_img = pd.DataFrame(image_json_list,columns=['item_id','img_name','label',
                                           'viewpoint','display','instance_id',
                                           'x_min','y_min','x_max','y_max',
                                               'img_path','img_text_path'])
print(df_img)
df_img.to_csv(dir_root + os.sep + 'Temp-File' + os.sep + 'img.csv')

    item_id img_name  label  viewpoint  display  instance_id  x_min  y_min  \
0    000001    0.jpg   长款外套        1.0      0.0   20000101.0  235.0   60.0   
1    000001    1.jpg   长款外套        0.0      0.0   20000101.0    3.0  195.0   
2    000001    2.jpg   长款外套        0.0      0.0   20000101.0  274.0  116.0   
3    000001    3.jpg   长款外套        2.0      0.0   20000101.0    0.0   71.0   
4    000001    4.jpg   None        NaN      NaN          NaN    NaN    NaN   
..      ...      ...    ...        ...      ...          ...    ...    ...   
108  000025    0.jpg  短袖Top        0.0      0.0   20002501.0  160.0   38.0   
109  000025    1.jpg  短袖Top        0.0      0.0   20002501.0  158.0   45.0   
110  000025    2.jpg   None        NaN      NaN          NaN    NaN    NaN   
111  000025    3.jpg   None        NaN      NaN          NaN    NaN    NaN   
112  000025    4.jpg   None        NaN      NaN          NaN    NaN    NaN   

      x_max   y_max                                           i

In [39]:
import cv2
import sys
# 把视频中的帧都抽取出来
for video_json in os.listdir(dir_video_annotation):
    # dir_ = dir_video + os.sep + video_json[:6] + ".mp4"
    cap = cv2.VideoCapture(dir_video + os.sep + video_json[:6] + ".mp4")
    dir_video_json = dir_video_annotation + os.sep + video_json
    f = open(dir_video_json,'r')
    video_dic = json.load(f)
    for frame in video_dic['frames']:
        cap.set(cv2.CAP_PROP_POS_FRAMES, float(frame['frame_index']))
        if cap.isOpened():
            if_success , spe_frame = cap.read()
            new_file_path =  dir_video_frame + os.sep + str(video_dic['video_id']) + "_" + str(frame['frame_index']) + ".jpg"
            # print(cv2.imencode('.jpg', spe_frame))
            cv2.imencode('.jpg', spe_frame)[1].tofile(new_file_path)
    cap.release()

In [38]:
#把video的json中的每个帧的annotation抽取出来
video_json_list = []
for video_json in os.listdir(dir_video_annotation):
    dir_video_json = dir_video_annotation + os.sep + video_json
    dir_frame = dir_video_frame + os.sep + video_json[:6] + "_"
    f = open(dir_video_json,'r')
    video_dic = json.load(f)
    for frame in video_dic['frames']:
        if len(frame['annotations']) != 0:
            for j in range(0,len(frame['annotations'])):
                video_json_list.append([video_dic['video_id'],frame['frame_index'],
                                    frame['annotations'][j]['label'],
                                    frame['annotations'][j]['viewpoint'],
                                    frame['annotations'][j]['display'],
                                    frame['annotations'][j]['instance_id'],
                                    frame['annotations'][j]['box'][0], 
                                    frame['annotations'][j]['box'][1],
                                    frame['annotations'][j]['box'][2],
                                    frame['annotations'][j]['box'][3],
                                    dir_frame+str(frame['frame_index'])+'.jpg',
                                    dir_video_text+os.sep+video_json[:6]+'.txt'])
        # elif len(frame['annotations']) == 0:
df_video = pd.DataFrame(video_json_list,columns=['video_id','frame_index','label',
                                                 'viewpoint','display','instance_id',
                                                 'x_min','y_min','x_max','y_max',
                                                 'frame_path','video_text_path'])
print(df_video)
df_video.to_csv(dir_root + os.sep + 'Temp-File' + os.sep + 'video.csv')

    video_id  frame_index  label  viewpoint  display  instance_id  x_min  \
0     000001            0   长款外套          0        1     20000101    214   
1     000001            0     长裤          0        1            0    289   
2     000001           40   长款外套          0        1     20000101    248   
3     000001           80   长款外套          0        1     20000101    128   
4     000001          120   长款外套          0        1     20000101    242   
..       ...          ...    ...        ...      ...          ...    ...   
463   000025          280     中裤          0        1            0    207   
464   000025          320  短袖Top          0        1     20002501    126   
465   000025          320     中裤          0        1            0    103   
466   000025          360  长袖Top          0        1     20002501    143   
467   000025          360     中裤          0        1            0    152   

     y_min  x_max  y_max                                         frame_path  \
0      5

In [2]:
inch = 2.54
meta = 14*inch/((16*16+9*9)**0.5)
print("14寸(16:9):长:"+str(meta*16)+"  高:"+str(meta*9))
print("面积:"+str(meta**2*16*9))

meta = 13*inch/((3*3+2*2)**0.5)
print("13寸(3:2):长:"+str(meta*3)+"  高:"+str(meta*2))
print("面积:"+str(meta**2*6))

meta = 15.6*inch/((16*16+9*9)**0.5)
print("15.6寸(16:9):长:"+str(meta*16)+"  高:"+str(meta*9))

meta = 16.1*inch/((16*16+9*9)**0.5)
print("16.1寸(16:9):长:"+str(meta*16)+"  高:"+str(meta*9))

14寸(16:9):长:30.993226100148974  高:17.433689681333796
面积:540.3262860534126
13寸(3:2):长:27.474300719035604  高:18.316200479357068
面积:503.2248000000001
15.6寸(16:9):长:34.535309083023144  高:19.42611135920052
16.1寸(16:9):长:35.64221001517132  高:20.04874313353387


In [6]:
import numpy as np
fc_np = np.array([]).reshape(-1, 64)
print(fc_np.shape)


(0, 64)
